<div style="background: url(https://www.ultimatebattle.in/wp-content/uploads/2018/09/pubg-events-1140x330.jpg);padding: 50px;background-size: contain;background-repeat: no-repeat;background-position: center;">
<center >
  <div style="background-coloe:#FFFFFF;"> 
      <h1 style="margin-top:220px; color:white;font-weight:500;font-size:1.5em;">A complete pipline for making predictions using fastai library</h1>
  </div>
</center>
</div>

## Index
* [Introduction](#Introduction)
* [Understanding the project](#Understanding the project)
* [Data Visualization](#Data Visualization)
* [Baseline model using fastai library](#Baseline model)
* [Validation & analysis](#Validation & Analysis)
* [Submit](Submit)

# Introduction
### About the notebook
This notebook aims to cover all the necessary information to design the pipline for the [Kaggle PUBG challenge](https://www.kaggle.com/c/pubg-finish-placement-prediction). In this notebook, I will try to explain you the fastai implementation using the fastai version 0.7.0. Though I may plan to use the recent fastai version in future notebook if I get a good response for this notebook. We will perform some EDA to understand the nature of data but will not peek into the data as this is not considered as a good practice while working with the data. I believe that this notion is kept in mind so that we do not create a model to overfit the data. If you feel that this requires any changes, feel free to comment and I will try to correct it ASAP.

### A little about PUBG data
We all love PUBG just like we love Data Science. When the team at [PUBG](https://www.pubg.com/) decided to make official game data available for the public to explore and scavenge outside of "The Blue Circle.", it became a great opportunity to look into the game stats and data of global users and play with the same.

> Kaggle mentions "This competition is not an official or affiliated PUBG site - Kaggle collected data made possible through the [PUBG Developer API](https://developer.pubg.com/)."

# Understanding the contest
We are provided with a large number of anonymized PUBG game stats, formatted so that each row contains one player's post-game stats. The data comes from matches of all types: *solos*, *duos*, *squads*, and *custom*; there is no guarantee of there being 100 players per match, nor at most 4 player per group.

We are required to predict the **"winPlacePerc"** which corresponds to the percentage prediction that a player is going to have a **Chicken Dinner** or not.

### Evaluation
The contest wants use to minimize the [Mean Absolute Error](https://en.wikipedia.org/wiki/Mean_absolute_error) given by the following formula $$MAE = \frac{\sum_{i=1}^{n} | y_i - x_i | }{n}$$

### Having a look at the data

In [ ]:
import pandas as pd
train = pd.read_csv('../input/train_V2.csv')
test = pd.read_csv('../input/test_V2.csv')
print(f"The training dataset contains {len(train.Id)} rows and {len(train.columns)} columns")
print(f"and the testing dataset contains {len(test.Id)} rows and {len(test.columns)} columns")
train.tail()

We can see that the *train_V2.csv* contains **4446965** rows of data each having **29** columns. and the *test_V2.csv* contains ** ** rows and **28** columns.

The data description does defines the meaning of each column but let me make it a bit redundant for you by mentioning it here itself so that you don't have to go back to the competition page:
### Data fields

*  **Id** - Player’s Id
* **groupId** - ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
* **matchId** - ID to identify match. There are no matches that are in both the training and testing set.
*  **assists** - Number of enemy players this player damaged that were killed by teammates.
*  **boosts** - Number of boost items used.
*  **damageDealt** - Total damage dealt. Note: Self inflicted damage is subtracted.
*  **DBNO	s** - Number of enemy players knocked.
*  **headshotKills** - Number of enemy players killed with headshots.
*  **heals** - Number of healing items used.
*  **killPlace** - Ranking in match of number of enemy players killed.
*  **killPoints** - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.) If there is a value other than -1 in rankPoints, then any 0 in killPoints should be treated as a “None”.
* **kills** - Number of enemy players killed.
*  **killStreaks** - Max number of enemy players killed in a short amount of time.
* **longestKill** - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
* **matchDuration** - Duration of match in seconds.
* **matchType** - String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches.
* **maxPlace** - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
* **numGroups** - Number of groups we have data for in the match.
* **rankPoints** - Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.
* **revives** - Number of times this player revived teammates.
* **rideDistance** - Total distance traveled in vehicles measured in meters.
* **roadKills** - Number of kills while in a vehicle.
* **swimDistance** - Total distance traveled by swimming measured in meters.
* **teamKills** - Number of times this player killed a teammate.
* **vehicleDestroys** - Number of vehicles destroyed.
* **walkDistance** - Total distance traveled on foot measured in meters.
* **weaponsAcquired** - Number of weapons picked up.
* **winPoints** - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.) If there is a value other than -1 in rankPoints, then any 0 in winPoints should be treated as a “None”.
* **winPlacePerc** - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.

# Data Visualization
We are now going to perform some EDA on the data just to get the gist of what kind of data we are dealing with. Here we will be plotting a random *Column* vs *winPlacePerc* to plot the data distribution w.r.t the required result. 

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams['agg.path.chunksize'] = 10000
%matplotlib inline
plt.style.use('seaborn')

In [ ]:
# cols=train.drop(columns=['Id','groupId','matchId','matchType','winPlacePerc']).columns
# alter=train['winPlacePerc']
# fig, axes = plt.subplots(5,4)
# plt.figure(figsize=(10, 15))
# idx=0
# for rows in axes:
#     for col in rows:
# #         print(idx, cols[idx], type(alter[0:5]),type(train[cols[idx]][0:5]))
#         col.plot(alter,train[cols[idx]], 'bo')
#         idx=idx+1

In [ ]:
!pip install fastai==0.7.0 --no-deps
!pip install torch==0.4.1

In [ ]:
from fastai.structured import *
from fastai.column_data import *

PATH='../input/'
!ls {PATH}

In [ ]:
train.describe()

In [ ]:
for col in train.columns:
    print({i:j for i,j in zip(np.unique(train[col].isnull(),return_counts=True)[0],np.unique(train[col].isnull(),return_counts=True)[1])})

In [ ]:
train.winPlacePerc.fillna(0, inplace=True)
for col in train.columns:
    print(col, {i:j for i,j in zip(np.unique(train[col].isnull(),return_counts=True)[0],np.unique(train[col].isnull(),return_counts=True)[1])})

In [ ]:
train.matchType.unique()

In [ ]:
val_idx=np.array(train.Id[-100000:].index)
print(len(val_idx))

In [ ]:
cat_vars=['matchType']
for v in cat_vars: train[v] = train[v].astype('category').cat.as_ordered()
apply_cats(test, train)

In [ ]:
train.matchType=test.matchType.cat.codes
test.matchType=test.matchType.cat.codes

In [ ]:
learn = ColumnarModelData.from_data_frame('./', val_idx, train.drop(columns=['Id','groupId','matchId', 'winPlacePerc']), 
                                      train.winPlacePerc, cat_flds=cat_vars, bs=128,
                                      test_df=test.drop(columns=['Id','groupId','matchId']))

In [ ]:
learner = learn.get_learner([],24,0.04, 1, [1000,500], [0.001,0.01])
learner.fit(1e-2, 1, cycle_len=3, metrics=[metrics.mean_absolute_error])

In [ ]:
learn.sched.plot()

In [ ]:
SAMPLE_SUB = '/kaggle/input/sample_submission.csv'
pred_t = learn.predict(is_test=True)
sample_df = pd.read_csv(SAMPLE_SUB)
sample_list = list(sample_df.Id)
pred_list = [p for p in preds_t]
df = pd.DataFrame({'Id':sample_list,'winPlacePerc':pred_list})
df.to_csv('submission.csv', header=True, index=False)